In [4]:
#Gated Convolutional Network with aspect Embedding (based on a CNN model)
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Concatenate, Multiply, Flatten

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load your dataset
df = pd.read_csv('/content/final_dataset - Sheet1.csv')

# Split the data into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Tokenize the comments
max_words = 10000
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(train_df['Comment'])

# Convert comments to sequences
train_sequences = tokenizer.texts_to_sequences(train_df['Comment'])
test_sequences = tokenizer.texts_to_sequences(test_df['Comment'])

# Pad sequences
max_sequence_length = max(len(seq) for seq in train_sequences)
X_train = pad_sequences(train_sequences, maxlen=max_sequence_length)
X_test = pad_sequences(test_sequences, maxlen=max_sequence_length)

# Prepare target data
y_train = np.array(train_df['Stance'])
y_test = np.array(test_df['Stance'])

# Prepare target-specific information
target_info_train = np.array(train_df['Target'])
target_info_test = np.array(test_df['Target'])

# Build GCAE model
embedding_dim = 100
aspect_embedding_dim = 20
num_filters = 64
filter_sizes = [3, 4, 5]

# Input layers
input_comment = Input(shape=(max_sequence_length,), name='input_comment')
input_target = Input(shape=(1,), name='input_target')

# Embedding layer for comments
embedding_layer = Embedding(input_dim=max_words, output_dim=embedding_dim)(input_comment)

# Convolutional layers
conv_layers = []
for filter_size in filter_sizes:
    conv_layer = Conv1D(num_filters, filter_size, activation='relu')(embedding_layer)
    pooled_layer = GlobalMaxPooling1D()(conv_layer)
    conv_layers.append(pooled_layer)

# Concatenate pooled convolutional layers
concatenated_layer = Concatenate()(conv_layers)

# Flatten the concatenated layer
flattened_layer = Flatten()(concatenated_layer)

# Aspect Embedding
aspect_embedding_layer = Embedding(input_dim=max_words, output_dim=aspect_embedding_dim)(input_target)

# Flatten the aspect embedding layer
flattened_aspect_embedding = Flatten()(aspect_embedding_layer)

# Ensure both flattened layers have the same number of dimensions
flattened_aspect_embedding = Dense(192)(flattened_aspect_embedding)

# Multiply flattened aspect embedding with flattened concatenated layer
gated_layer = Multiply()([flattened_layer, flattened_aspect_embedding])

# Dense layers
dense_layer = Dense(32, activation='relu')(gated_layer)
output_layer = Dense(1, activation='sigmoid', name='output')(dense_layer)

# Build and compile the model
model = Model(inputs=[input_comment, input_target], outputs=output_layer)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit([X_train, target_info_train], y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
y_pred = model.predict([X_test, target_info_test])
y_pred_binary = (y_pred > 0.5).astype(int)

# Overall metrics
test_loss, test_accuracy = model.evaluate([X_test, target_info_test], y_test)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy * 100:.2f}%')

# Calculate metrics for each target class
unique_targets = df['Target'].unique()

for target_class in unique_targets:
    indices = test_df[test_df['Target'] == target_class].index
    indices = indices[indices < len(y_test)]  # Ensure indices are within bounds

    y_test_target = y_test[indices]
    y_pred_target = y_pred_binary[indices]

    accuracy = accuracy_score(y_test_target, y_pred_target)
    precision = precision_score(y_test_target, y_pred_target)
    recall = recall_score(y_test_target, y_pred_target)
    f1 = f1_score(y_test_target, y_pred_target)

    print(f'Target {target_class} Metrics:')
    print(f'  Accuracy: {accuracy * 100:.2f}%')
    print(f'  Precision: {precision:.4f}')
    print(f'  Recall: {recall:.4f}')
    print(f'  F1 Score: {f1:.4f}')

Epoch 1/10
110/110 [==============================] - 36s 279ms/step - loss: 0.5734 - accuracy: 0.6896 - val_loss: 0.5125 - val_accuracy: 0.7079
Epoch 2/10
110/110 [==============================] - 28s 255ms/step - loss: 0.4671 - accuracy: 0.7440 - val_loss: 0.3942 - val_accuracy: 0.8362
Epoch 3/10
110/110 [==============================] - 28s 256ms/step - loss: 0.2081 - accuracy: 0.9192 - val_loss: 0.3002 - val_accuracy: 0.8877
Epoch 4/10
110/110 [==============================] - 28s 251ms/step - loss: 0.0612 - accuracy: 0.9811 - val_loss: 0.3026 - val_accuracy: 0.9015
Epoch 5/10
110/110 [==============================] - 28s 253ms/step - loss: 0.0179 - accuracy: 0.9966 - val_loss: 0.3439 - val_accuracy: 0.9038
Epoch 6/10
110/110 [==============================] - 28s 255ms/step - loss: 0.0094 - accuracy: 0.9986 - val_loss: 0.3742 - val_accuracy: 0.9003
Epoch 7/10
110/110 [==============================] - 28s 258ms/step - loss: 0.0072 - accuracy: 0.9986 - val_loss: 0.3910 - val_ac

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and bei